# Machinations to CadCAD conversion guide 1 
# for Infinite runner model

https://machinations.io/docs/tutorials-examples/infinite-runner-game-loop/

## Importing libraries

In [570]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
pd.options.plotting.backend = "plotly"
from dataclasses import dataclass
import copy
from dataclasses import field
from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend

## Defining the types of variables

In [571]:
METERS = int
COINS = int
PERCENTAGE = float   

## Defining the System Params

In [572]:
#utils
def default(obj):
    return field(default_factory=lambda: copy.copy(obj))


@dataclass
class Parameters:
    # crash_chance is the chance of crashing in the beginning
    crash_chance: PERCENTAGE = default([10])

    # crash_increase is by how much the difficulty_factor will increase each timestep
    crash_increase: PERCENTAGE = default([1])


# Initialize Parameters instance with default values
system_params = Parameters().__dict__


## State Variables

In [573]:
@dataclass
class StateVariables:
    distance: METERS = 0
    coins: COINS = 0
    difficulty_factor: int =0
    player_crashes: int = 0

initial_state = StateVariables().__dict__

## Policy Functions

In [574]:
def p_sprint(params, substep, state_history, prev_state, **kwargs):
    '''Calculates the amount of distance covered per timestep'''

    if prev_state['player_crashes']==1:
        distance_covered = 0
    else:
        distance_covered=5
    
    return {'distance_covered': distance_covered}

def p_difficulty(params, substep, state_history, prev_state, **kwargs):
    '''Calculates the increase in difficulty every timestep'''

    if prev_state['timestep']<1:
        difficulty_increase=0

    # if player crashed in previous step then dont increase difficulty factor
    elif prev_state['player_crashes'] == 1:
        difficulty_increase=0
    
    else:
        #Every second timestep increase difficulty by 1
        difficulty_increase = 1 if prev_state['timestep']%2==1 else 0

    return {'difficulty_increase': difficulty_increase}


def p_generate_coins(params, substep, state_history, prev_state, **kwargs):
    '''Calculates the amount of coins generated'''

    if prev_state['distance']<1:
        new_coins=0

    # if player crashed in previous step then dont increase coins
    elif prev_state['player_crashes'] == 1:
        new_coins=0
    
    else:
        # if distance is less than 50 mint 1 coin, if its less than 100 mint 2, if its above 100 mint 3
        distance = prev_state['distance']
        new_coins = 1 if distance < 50 else 2 if distance < 100 else 3

    return {'new_coins': new_coins}


def p_crash(params, substep, state_history, prev_state, **kwargs):
    '''Calculates the probability of crash'''

    if prev_state['difficulty_factor']<2:
        player_crashed=0

    elif prev_state['player_crashes']==1:
        player_crashed=1
    
    else: 
        # Take the initial chance of crashing and adding it with the current difficulty factor to update the chance of crash
        crash_chance = (params['crash_chance'] + prev_state['difficulty_factor'])/100
        # If the random number generated between 0 and 1 is smaller than the percentage chance of crashing then we assume the player crashed
        if random.random()< crash_chance:
            player_crashed = 1 
        else:
            player_crashed=0

    return {'player_crashed': player_crashed}


## State Update Functions and Partial State Update Blocks

In [575]:

def s_update_distance(params, substep, state_history, prev_state, policy_input, **kwargs):
    '''Update the state of the distance variable by the amount of distance sprinted'''

    updated_distance = np.ceil(prev_state['distance'] + policy_input['distance_covered'])
    return ('distance', max(updated_distance, 0))


def s_update_coins(params, substep, state_history, prev_state, policy_input, **kwargs):
    '''Update the state of the coins variable by the amount of new coins generated'''

    updated_coins = np.ceil(prev_state['coins'] + policy_input['new_coins'])
    return ('coins', max(updated_coins, 0))


def s_update_difficulty(params, substep, state_history, prev_state, policy_input, **kwargs):
    '''Update the state of the difficulty variable by the amount of difficulty increase'''

    updated_difficulty = np.ceil(prev_state['difficulty_factor'] + policy_input['difficulty_increase'])

    return ('difficulty_factor', max(updated_difficulty, 0))

def s_update_crash(params, substep, state_history, prev_state, policy_input, **kwargs):
    '''Update the state of the crash variable'''

    updated_crash = policy_input['player_crashed']

    return ('player_crashes', max(updated_crash, 0))

###

state_update_blocks = [
    {
        'policies': {
            'p_sprint': p_sprint,
            'p_difficulty': p_difficulty,
            'p_generate_coins':p_generate_coins,
            'p_crash':p_crash,
        },
        'variables': {
            'distance': s_update_distance,
            'coins': s_update_coins,
            'difficulty_factor':s_update_difficulty,
            'player_crashes': s_update_crash

        }
    },

]



In [576]:
# config and run

#number of timesteps
TIMESTEPS = 40
#number of monte carlo runs
RUNS = 5


model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)

experiment = Experiment(simulation)
# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS)

result = experiment.run()


df = pd.DataFrame(result)
df

,distance,coins,difficulty_factor,player_crashes,simulation,subset,run,substep,timestep
0,0.0,0.0,0.0,0,0,0,1,0,0
1,5.0,0.0,0.0,0,0,0,1,1,1
2,10.0,1.0,1.0,0,0,0,1,1,2
3,15.0,2.0,1.0,0,0,0,1,1,3
4,20.0,3.0,2.0,0,0,0,1,1,4
...,...,...,...,...,...,...,...,...,...
200,30.0,5.0,3.0,1,0,0,5,1,36
201,30.0,5.0,3.0,1,0,0,5,1,37
202,30.0,5.0,3.0,1,0,0,5,1,38
203,30.0,5.0,3.0,1,0,0,5,1,39


## Let's plot these 5 monte carlo runs

In [577]:
fig = px.line(
    df,
    x='timestep',
    y=['coins', 'distance'],
    facet_col='run',
    height=400,
    template='seaborn',
    title='Coins and distance plotted for different runs'
)

fig.update_layout(
    margin=dict(l=20, r=200, t=100, b=20),
)

fig.show()

In [578]:
# final coins and distance in every run
df2 = df.groupby(['run'])[['coins','distance']].max()
df2

,coins,distance
run,,
1,19.0,75.0
2,4.0,25.0
3,65.0,160.0
4,19.0,75.0
5,5.0,30.0


In [579]:
# final coins distance and crash timestep for each run
first_crash = df[df['player_crashes'] == 1].groupby(['run'])['timestep'].min()
df2['crash_timestep'] = first_crash.values
df2


,coins,distance,crash_timestep
run,,,
1,19.0,75.0,15
2,4.0,25.0,5
3,65.0,160.0,32
4,19.0,75.0,15
5,5.0,30.0,6


## Checking if data matches machinations simulations

In [610]:
# config and run

#number of timesteps
TIMESTEPS = 100
#number of monte carlo runs
RUNS = 100


model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)

experiment = Experiment(simulation)
# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS)

result = experiment.run()


df = pd.DataFrame(result)
df

,distance,coins,difficulty_factor,player_crashes,simulation,subset,run,substep,timestep
0,0.0,0.0,0.0,0,0,0,1,0,0
1,5.0,0.0,0.0,0,0,0,1,1,1
2,10.0,1.0,1.0,0,0,0,1,1,2
3,15.0,2.0,1.0,0,0,0,1,1,3
4,20.0,3.0,2.0,0,0,0,1,1,4
...,...,...,...,...,...,...,...,...,...
10095,40.0,7.0,4.0,1,0,0,100,1,96
10096,40.0,7.0,4.0,1,0,0,100,1,97
10097,40.0,7.0,4.0,1,0,0,100,1,98
10098,40.0,7.0,4.0,1,0,0,100,1,99


In [611]:
# final coins and distance in every run
df2 = df.groupby(['run'])[['coins','distance']].max()
# final coins distance and crash timestep for each run
first_crash = df[df['player_crashes'] == 1].groupby(['run'])['timestep'].min()
df2['crash_timestep'] = first_crash.values
df2.describe()

,coins,distance,crash_timestep
count,100.000000,100.000000,100.000000
mean,13.970000,56.750000,11.350000
std,12.975077,31.737361,6.347472
min,4.000000,25.000000,5.000000
25%,6.000000,35.000000,7.000000
50%,8.000000,45.000000,9.000000
75%,17.000000,70.000000,14.000000
max,65.000000,160.000000,32.000000


In [612]:
# comparing with our machinations simulation results
machinations_results = pd.read_csv('data\inifite_runner_machinations100.csv')

# Data cleaning and processing
array=[]
for i in machinations_results['play'].unique():
    if i ==1:
        pass
    else:
        array.append(dict(machinations_results[machinations_results['play']==i].iloc[-1]))

machinations_df = pd.DataFrame(array)
machinations_df[['Coins / 9','Distance / 8', 'step']].describe()
machinations_df.rename(columns = {'Coins / 9':'coins', 'Distance / 8':'distance',
                              'step':'crash_timestep'}, inplace = True)


In [613]:
machinations_df.head(10)

,play,crash_timestep,distance,coins,Player Crashes / 10,Difficulty Factor / 11
0,2.0,3.0,15.0,2.0,0.0,1.0
1,111.0,13.0,65.0,15.0,1.0,6.0
2,112.0,17.0,85.0,23.0,1.0,8.0
3,113.0,17.0,85.0,23.0,1.0,8.0
4,114.0,5.0,25.0,4.0,1.0,2.0
5,115.0,7.0,35.0,6.0,1.0,3.0
6,116.0,13.0,65.0,15.0,1.0,6.0
7,117.0,11.0,55.0,11.0,1.0,5.0
8,118.0,8.0,40.0,7.0,1.0,4.0
9,119.0,12.0,60.0,13.0,1.0,6.0


In [614]:
# Means of coins distance and crash_timestep column in CadCAD simulation

df2[['coins','distance','crash_timestep']].mean()

coins             13.97
distance          56.75
crash_timestep    11.35
dtype: float64

In [615]:
# Means of coins distance and crash_timestep column in Machinations simulation

machinations_df[['coins','distance','crash_timestep']].mean()

coins             13.796610
distance          57.966102
crash_timestep    11.593220
dtype: float64

In [616]:
error = df2[['coins','distance','crash_timestep']].mean() - machinations_df[['coins','distance','crash_timestep']].mean()

The means of all columns for 100 runs look similar so it seems like we succesfully converted the model

# Analysis

### Let's check when the first crash happens for each subset and run

In [617]:
first_crash = df[df['player_crashes'] == 1].groupby(['subset', 'run'])['timestep'].min().reset_index()

In [618]:
fig = px.bar(first_crash, x='run', y='timestep', facet_col='subset', title='Player crashed timestep by Subset and Run')
fig.show()


In [637]:
fig = px.histogram(first_crash, x='timestep', title='Player crashed timestep histogram', nbins=30)
fig.show()

In [619]:
first_crash

,subset,run,timestep
0,0,1,5
1,0,2,5
2,0,3,11
3,0,4,15
4,0,5,16
...,...,...,...
95,0,96,13
96,0,97,8
97,0,98,5
98,0,99,22


### Let's run a parameter sweep on crash chance from 10% to 60% and see how 20 monte carlo runs on each play out

In [589]:


# config and run
TIMESTEPS = 50
RUNS = 6
model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)
simulation.model.params.update({
    # Running a parameter sweep on the initial chance of crashing 
    'crash_chance':[10,30,60]
})
experiment = Experiment(simulation)

# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS)

result = experiment.run()


df = pd.DataFrame(result)
df

,distance,coins,difficulty_factor,player_crashes,simulation,subset,run,substep,timestep
0,0.0,0.0,0.0,0,0,0,1,0,0
1,5.0,0.0,0.0,0,0,0,1,1,1
2,10.0,1.0,1.0,0,0,0,1,1,2
3,15.0,2.0,1.0,0,0,0,1,1,3
4,20.0,3.0,2.0,0,0,0,1,1,4
...,...,...,...,...,...,...,...,...,...
913,35.0,6.0,3.0,1,0,2,6,1,46
914,35.0,6.0,3.0,1,0,2,6,1,47
915,35.0,6.0,3.0,1,0,2,6,1,48
916,35.0,6.0,3.0,1,0,2,6,1,49


In [590]:
# checking when the first crash happened in each subset and run
first_crash = df[df['player_crashes'] == 1].groupby(['subset', 'run'])['timestep'].min().reset_index()

In [591]:
# plotting the timestep for each crash
fig = px.bar(first_crash, x='run', y='timestep', facet_col='subset', title='Player crashed timestep by Subset and Run')
fig.show()

As we increase the crash chance its clear the timestep at which on average seems to decrease let's prove this by calculating the mean

In [592]:
subset_means = first_crash.groupby('subset')['timestep'].mean()

# Create bar chart
fig = px.bar(subset_means, x=subset_means.index, y=subset_means.values, title='Mean First Change by Subset')

# Create line graph
line_trace = go.Scatter(x=subset_means.index, y=subset_means.values, mode='lines', name='Mean First Change')

# Add line trace to bar chart
fig.add_trace(line_trace)

# Show plot
fig.show()


In [593]:
fig = px.line(
    df,
    x='timestep',
    y=['coins', 'distance'],
    facet_row='subset',
    facet_col='run',
    height=800,
    template='seaborn',
    title='Distance plotted for different runs and subsets'
)

fig.update_layout(
    margin=dict(l=100, r=100, t=100, b=100),
)

fig.show()

In [594]:
# on average when does the crash happen for each run
first_crash = df[df['player_crashes'] == 1].groupby(['subset', 'run'])['timestep'].min().reset_index()
subset_means = first_crash.groupby('subset')['timestep'].mean()
subset_means

subset
0    10.000000
1     6.500000
2     5.333333
Name: timestep, dtype: float64

In [595]:
descriptive_stats = df.groupby(['subset']).agg(['mean', 'median', 'std', 'min', 'max'])
descriptive_stats['distance']

,mean,median,std,min,max
subset,,,,,
0,43.823529,40.0,21.407557,0.0,85.0
1,30.032680,30.0,9.170457,0.0,40.0
2,24.983660,25.0,6.446480,0.0,35.0


In [596]:
descriptive_stats['coins']

,mean,median,std,min,max
subset,,,,,
0,9.013072,7.0,6.339041,0.0,23.0
1,5.026144,5.0,1.773938,0.0,7.0
2,4.016340,4.0,1.218597,0.0,6.0


## Scenario Analysis

Let's say the game team has a shop in the game where certain items cost 50 coins. And they believe that the median player should have to play 10 games to earn these items to adequently incentivize them. What should be the initial value of the crash chance parameter to get 5 as the median coins

In [597]:
TIMESTEPS = 30
RUNS = 100

model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)

def median_coin_100_runs(i):
    simulation.model.params.update({
        # Running a parameter sweep on the initial chance of crashing 
        'crash_chance':[i]
    })
    experiment = Experiment(simulation)

    # Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
    experiment.engine = Engine(backend=Backend.PATHOS)

    result = experiment.run()


    df = pd.DataFrame(result)

    median_coins = df['coins'].median()
    return median_coins



In [598]:
median_coins_expected = 5
crash_chance_initial_guess = 1
step_distance = 3


guess = crash_chance_initial_guess

for i in range(20):
    median_for_guess = median_coin_100_runs(guess)
    print(i,', guess = ',guess, ', median_for_guess = ', median_for_guess)
    if median_for_guess>median_coins_expected:
        guess+=step_distance
    else:
        break

print(guess)

0 , guess =  1 , median_for_guess =  13.0
1 , guess =  4 , median_for_guess =  8.0
2 , guess =  7 , median_for_guess =  7.0
3 , guess =  10 , median_for_guess =  7.0
4 , guess =  13 , median_for_guess =  6.0
5 , guess =  16 , median_for_guess =  6.0
6 , guess =  19 , median_for_guess =  5.0
19


In [599]:
guess 

19

Therefore to get a median coin value of 5 we need crash_chance parameter = 19